In [2]:
from typing import List
import pydicom
from pydicom.dataset import Dataset
from io import BytesIO
import os 
import nibabel as nib
import numpy as np
import torch
from rt_utils import RTStructBuilder

In [5]:
rtstruct_path = "/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/META/"

In [7]:
def load_dicom_datasets(dicom_folder_path: str) -> List[Dataset]:
    """
    Charge tous les fichiers DICOM d'un dossier donné en datasets pydicom.
    """
    dicom_datasets = []
    for filename in os.listdir(dicom_folder_path):
        if filename.endswith('.dcm'):
            file_path = os.path.join(dicom_folder_path, filename)
            try:
                ds = pydicom.dcmread(file_path)
                dicom_datasets.append(ds)
            except Exception as e:
                print(f"Erreur lors de la lecture du fichier DICOM {filename}: {e}")
                continue  # ou lever une exception selon les besoins de votre application

    if not dicom_datasets:
        raise Exception("Aucun fichier DICOM valide trouvé dans le dossier spécifié.")

    return dicom_datasets

rtstruct_file = load_dicom_datasets(rtstruct_path)
print(rtstruct_file[0])

rtstruct = rtstruct_file[0]

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 278
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: RT Structure Set Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.752.243.1.1.20230602094342956.1800.51115
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.1.2.100.8.40.1101.0
(0002, 0013) Implementation Version Name         SH: 'DicomObjects.NET'
(0002, 0100) Private Information Creator UID     UI: 1.2.826.0.1.3680043.1.2.100.8.40.1101.0
(0002, 0102) Private Information                 OB: b'DicomObjects.NET'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0012) Instance Creation Date              DA: '20230602'
(0008, 0013) Instance Creation Time              TM: '094342.0

In [9]:
print(type(rtstruct))

<class 'pydicom.dataset.FileDataset'>


# Récupérer les diamètres et les volumes des rois

In [22]:
import os
import pydicom
import numpy as np
from shapely.geometry import Polygon
from dicompylercore import dicomparser

# Charger le fichier DICOM RTStruct
rtstruct_path = "/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/META/RS1.2.752.243.1.1.20230602094342956.1800.51115.dcm"
dicoms_path = "/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/RM/IRM"

# Charger les images de la série DICOM
dicom_files = [os.path.join(dicoms_path, f) for f in os.listdir(dicoms_path) if f.endswith('.dcm')]
dicom_series = [pydicom.dcmread(f) for f in dicom_files]
dicom_series.sort(key=lambda x: int(x.InstanceNumber))

# Extraire les informations de PixelSpacing et SliceThickness
pixel_spacing = dicom_series[0].PixelSpacing
slice_thickness = dicom_series[0].SliceThickness

# Charger le fichier RTStruct avec dicompyler-core
rtstruct = dicomparser.DicomParser(rtstruct_path)

# Extraire les structures
structures = rtstruct.GetStructures()

# Fonction pour calculer le diamètre d'un ROI
def calculate_diameter(contour):
    max_distance = 0
    for i in range(len(contour)):
        for j in range(i + 1, len(contour)):
            distance = np.linalg.norm(np.array(contour[i]) - np.array(contour[j]))
            if distance > max_distance:
                max_distance = distance
    return max_distance

# Fonction pour calculer le volume d'un ROI
def calculate_volume(coords, thickness):
    volume = 0
    for z in coords.keys():
        contours = coords[z]
        for contour in contours:
            polygon = Polygon(contour['data'])
            volume += polygon.area * thickness
    return volume / 1000  # Convertir en cm³

# Parcourir les structures et extraire les informations
for roi_number, roi_data in structures.items():
    roi_name = roi_data['name']
    print(f"ROI: {roi_name}")
    
    # Obtenir les coordonnées des contours
    coords = rtstruct.GetStructureCoordinates(roi_number)
    
    # Calculer le volume
    thickness = dicomparser.DicomParser.CalculatePlaneThickness(rtstruct, coords)
    volume = calculate_volume(coords, thickness)
    
    # Calculer le diamètre maximal
    contours = []
    for plane in coords.values():
        for contour in plane:
            contours.append(contour['data'])
    diameters = [calculate_diameter(contour) for contour in contours]
    
    # Obtenir les slices de début et de fin
    start_slices = [contour[0][2] for contour in contours if contour]
    end_slices = [contour[-1][2] for contour in contours if contour]

    print(f"  Diamètre max: {max(diameters):.2f} mm")
    print(f"  Volume: {volume:.2f} cm³")
    print(f"  Slice de début: {min(start_slices)}")
    print(f"  Slice de fin: {max(end_slices)}")

ROI: GTV 1
  Diamètre max: 6.20 mm
  Volume: 0.14 cm³
  Slice de début: 47.77951
  Slice de fin: 53.77951
ROI: GTV2
  Diamètre max: 3.96 mm
  Volume: 0.05 cm³
  Slice de début: 56.77951
  Slice de fin: 60.77951
ROI: GTV3
  Diamètre max: 7.35 mm
  Volume: 0.18 cm³
  Slice de début: 12.77951
  Slice de fin: 18.77951
ROI: GTV4
  Diamètre max: 5.49 mm
  Volume: 0.11 cm³
  Slice de début: 12.77951
  Slice de fin: 17.77951
ROI: GTV 5
  Diamètre max: 4.69 mm
  Volume: 0.08 cm³
  Slice de début: -12.22049
  Slice de fin: -8.220492
ROI: gtv6
  Diamètre max: 20.11 mm
  Volume: 2.85 cm³
  Slice de début: 35.77951
  Slice de fin: 53.77951


# Pipeline fonctionnel OMG

In [28]:
import os
import pydicom
import numpy as np
from shapely.geometry import Polygon
from dicompylercore import dicomparser

# Charger le fichier DICOM RTStruct
rtstruct_path = "/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/META/RS1.2.752.243.1.1.20230602094342956.1800.51115.dcm"
dicoms_path = "/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/RM/IRM"

# Charger les images de la série DICOM
dicom_files = [os.path.join(dicoms_path, f) for f in os.listdir(dicoms_path) if f.endswith('.dcm')]
dicom_series = [pydicom.dcmread(f) for f in dicom_files]
dicom_series.sort(key=lambda x: int(x.InstanceNumber))

# Extraire les informations de PixelSpacing et SliceThickness
pixel_spacing = dicom_series[0].PixelSpacing
slice_thickness = dicom_series[0].SliceThickness

# Créer une map des positions de slices à leurs indices
slice_positions = {round(dcm.ImagePositionPatient[2], 2): i+1 for i, dcm in enumerate(dicom_series)}

# Charger le fichier RTStruct avec dicompyler-core
rtstruct = dicomparser.DicomParser(rtstruct_path)

# Extraire les structures
structures = rtstruct.GetStructures()

# Fonction pour calculer le diamètre d'un ROI
def calculate_diameter(contour):
    max_distance = 0
    for i in range(len(contour)):
        for j in range(i + 1, len(contour)):
            distance = np.linalg.norm(np.array(contour[i]) - np.array(contour[j]))
            if distance > max_distance:
                max_distance = distance
    return max_distance

# Fonction pour calculer le volume d'un ROI
def calculate_volume(coords, thickness):
    volume = 0
    for z in coords.keys():
        contours = coords[z]
        for contour in contours:
            polygon = Polygon(contour['data'])
            volume += polygon.area * thickness
    return volume / 1000  # Convertir en cm³

# Parcourir les structures et extraire les informations
for roi_number, roi_data in structures.items():
    roi_name = roi_data['name']
    print(f"ROI: {roi_name}")
    
    # Obtenir les coordonnées des contours
    coords = rtstruct.GetStructureCoordinates(roi_number)
    
    # Calculer le volume
    thickness = dicomparser.DicomParser.CalculatePlaneThickness(rtstruct, coords)
    volume = calculate_volume(coords, thickness)
    
    # Calculer le diamètre maximal
    contours = []
    contour_slice_indices = []
    for plane in coords.values():
        for contour in plane:
            contours.append(contour['data'])
            z_pos = round(contour['data'][0][2], 2)
            if z_pos in slice_positions:
                contour_slice_indices.append(slice_positions[z_pos])
    
    diameters = [calculate_diameter(contour) for contour in contours]
    nb_dicoms = len(dicom_series)
    # Obtenir les slices de début et de fin
    start_slice = nb_dicoms - max(contour_slice_indices) if contour_slice_indices else None
    end_slice = nb_dicoms - min(contour_slice_indices) if contour_slice_indices else None
    
    print(f"  Diamètre max: {max(diameters):.2f} mm")
    print(f"  Volume: {volume:.2f} cm³")
    print(f"  Slice de début: {start_slice}")
    print(f"  Slice de fin: {end_slice}")

ROI: GTV 1
  Diamètre max: 6.20 mm
  Volume: 0.14 cm³
  Slice de début: 30
  Slice de fin: 36
ROI: GTV2
  Diamètre max: 3.96 mm
  Volume: 0.05 cm³
  Slice de début: 23
  Slice de fin: 27
ROI: GTV3
  Diamètre max: 7.35 mm
  Volume: 0.18 cm³
  Slice de début: 65
  Slice de fin: 71
ROI: GTV4
  Diamètre max: 5.49 mm
  Volume: 0.11 cm³
  Slice de début: 66
  Slice de fin: 71
ROI: GTV 5
  Diamètre max: 4.69 mm
  Volume: 0.08 cm³
  Slice de début: 92
  Slice de fin: 96
ROI: gtv6
  Diamètre max: 20.11 mm
  Volume: 2.85 cm³
  Slice de début: 30
  Slice de fin: 48


In [27]:
print(len(dicom_series))

208


# Version API (ça marche !!!!)

In [43]:
import os
import pydicom
import numpy as np
from shapely.geometry import Polygon
from dicompylercore import dicomparser

def extract_roi_info(rtstruct, dicom_series):

    dicom_series.sort(key=lambda x: int(x.InstanceNumber))
    
    # Extraire les informations de PixelSpacing et SliceThickness
    pixel_spacing = dicom_series[0].PixelSpacing
    slice_thickness = dicom_series[0].SliceThickness

    # Créer une map des positions de slices à leurs indices
    slice_positions = {round(dcm.ImagePositionPatient[2], 2): i+1 for i, dcm in enumerate(dicom_series)}

    # Charger le fichier RTStruct avec dicompyler-core
    rtstruct = dicomparser.DicomParser(rtstruct)

    # Extraire les structures
    structures = rtstruct.GetStructures()

    # Fonction pour calculer le diamètre d'un ROI
    def calculate_diameter(contour):
        max_distance = 0
        for i in range(len(contour)):
            for j in range(i + 1, len(contour)):
                distance = np.linalg.norm(np.array(contour[i]) - np.array(contour[j]))
                if distance > max_distance:
                    max_distance = distance
        return max_distance

    # Fonction pour calculer le volume d'un ROI
    def calculate_volume(coords, thickness):
        volume = 0
        for z in coords.keys():
            contours = coords[z]
            for contour in contours:
                polygon = Polygon(contour['data'])
                volume += polygon.area * thickness
        return volume / 1000  # Convertir en cm³

    # Initialiser le dictionnaire de résultats
    roi_info = {}

    # Parcourir les structures et extraire les informations
    for roi_number, roi_data in structures.items():
        roi_name = roi_data['name']
        
        # Obtenir les coordonnées des contours
        coords = rtstruct.GetStructureCoordinates(roi_number)
        
        # Calculer le volume
        thickness = dicomparser.DicomParser.CalculatePlaneThickness(rtstruct, coords)
        volume = calculate_volume(coords, thickness)
        
        # Calculer le diamètre maximal
        contours = []
        contour_slice_indices = []
        for plane in coords.values():
            for contour in plane:
                contours.append(contour['data'])
                z_pos = round(contour['data'][0][2], 2)
                if z_pos in slice_positions:
                    contour_slice_indices.append(slice_positions[z_pos])
        
        diameters = [calculate_diameter(contour) for contour in contours]
        nb_dicoms = len(dicom_series)
        
        # Obtenir les slices de début et de fin
        start_slice = nb_dicoms - max(contour_slice_indices) if contour_slice_indices else None
        end_slice = nb_dicoms - min(contour_slice_indices) if contour_slice_indices else None

        # Ajouter les informations au dictionnaire
        roi_info[roi_name] = {
            "diameter_max": max(diameters),
            "volume_cm3": volume,
            "start_slice": start_slice,
            "end_slice": end_slice
        }
    
    return roi_info

###########################################################################################################################
# Simulation de rtstruct et dicoms chargés en mémoire en objet pydicom :
def load_dicom_datasets(dicom_folder_path: str) -> List[Dataset]:
    """
    Charge tous les fichiers DICOM d'un dossier donné en datasets pydicom.
    """
    dicom_datasets = []
    for filename in os.listdir(dicom_folder_path):
        if filename.endswith('.dcm'):
            file_path = os.path.join(dicom_folder_path, filename)
            try:
                ds = pydicom.dcmread(file_path)
                dicom_datasets.append(ds)
            except Exception as e:
                print(f"Erreur lors de la lecture du fichier DICOM {filename}: {e}")
                continue  # ou lever une exception selon les besoins de votre application

    if not dicom_datasets:
        raise Exception("Aucun fichier DICOM valide trouvé dans le dossier spécifié.")

    return dicom_datasets
rtstruct_path = "/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/META/"
dicom_files = load_dicom_datasets(dicoms_path)
rtstruct_files = load_dicom_datasets(rtstruct_path)
rtstruct = rtstruct_files[0]

print("type de rtstruct : ", type(rtstruct))
print("type de dicom_files", type(dicom_files)," ", type(dicom_files[0]))
###########################################################################################################################

roi_info = extract_roi_info(rtstruct, dicom_files)
print(roi_info)

type de rtstruct :  <class 'pydicom.dataset.FileDataset'>
type de dicom_files <class 'list'>   <class 'pydicom.dataset.FileDataset'>
{'GTV 1': {'diameter_max': 6.195562862894834, 'volume_cm3': 0.13796970097221614, 'start_slice': 30, 'end_slice': 36}, 'GTV2': {'diameter_max': 3.963352447342527, 'volume_cm3': 0.052886430403313006, 'start_slice': 23, 'end_slice': 27}, 'GTV3': {'diameter_max': 7.348493565303027, 'volume_cm3': 0.1795430703734, 'start_slice': 65, 'end_slice': 71}, 'GTV4': {'diameter_max': 5.488227596774759, 'volume_cm3': 0.10696379318934995, 'start_slice': 66, 'end_slice': 71}, 'GTV 5': {'diameter_max': 4.6931767726455815, 'volume_cm3': 0.0793605851773, 'start_slice': 92, 'end_slice': 96}, 'gtv6': {'diameter_max': 20.10715011343477, 'volume_cm3': 2.8540769060405506, 'start_slice': 30, 'end_slice': 48}}
